In [59]:
import numpy as np
import pandas as pd
import scipy.stats as stats 
import matplotlib.pyplot as plt  #Пакет для построения графиков и прочего
import seaborn as sns
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nbor9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nbor9\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Задание 1
Вернемся к набору данных о видеоиграх.

In [28]:
games = pd.read_csv("vgsales.csv")
games = games.dropna(axis='index', how='any', subset=['Critic_Score'])
games.head()


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,65.0,8.5,431.0,Nintendo,E
7,Wii Play,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,41.0,6.6,129.0,Nintendo,E


Для каждого вопроса:

- сформулируйте нулевую и альтернативную гипотезы;
- выберите пороговый уровень статистической значимости;
- опишите полученные результаты статистического теста.

### 1) Как критики относятся к спортивным играм?


In [33]:
print("Нулевая гипотеза: Критикам спортивные игры нравятся больше остальных\nАльтернативная гипотеза: Критикам нравятся спортивные игры не больше остальных\nУровень статистической значимости 0.05")
Sport = games[(games['Genre'] == 'Sports')]
Other = games[(games['Genre'] != 'Sports')]
statistic, pvalue = stats.ttest_1samp(Sport['Critic_Score'], Other['Critic_Score'].mean())

print(statistic,pvalue)
if(pvalue < 0.05):
    print("Не отвергаем нулевую гипотезу, pvalue = ", pvalue)
else:
    print("Отвергаем нулевую гипотезу, pvalue = ", pvalue)

Нулевая гипотеза: Критикам спортивные игры нравятся больше остальных
Альтернативная гипотеза: Критикам нравятся спортивные игры не больше остальных
Уровень статистической значимости 0.05
8.755317599633457 6.838888719771996e-18
Не отвергаем нулевую гипотезу, pvalue =  6.838888719771996e-18


### 2) Критикам нравятся больше игры на PC или на PS4?

In [39]:
print("Нулевая гипотеза: Критикам нравятся игры на PS4 нравятся больше чем на PC\nАльтернативная гипотеза: Критикам игры на PC нравятся больше чем на PS4\nУровень статистической значимости 0.05")
PC = games[(games['Platform'] == 'PC')]
PS4 = games[(games['Platform'] == 'PS4')]
statistic, pvalue = stats.ttest_ind(PS4.Critic_Score,PC.Critic_Score, equal_var=False)

print(statistic,pvalue)
if(pvalue < 0.05):
    print("Не отвергаем нулевую гипотезу, pvalue = ", pvalue)
else:
    print("Отвергаем нулевую гипотезу, pvalue = ", pvalue)

Нулевая гипотеза: Критикам нравятся игры на PS4 нравятся больше чем на PC
Альтернативная гипотеза: Критикам игры на PC нравятся больше чем на PS4
Уровень статистической значимости 0.05
-4.3087588262138725 2.067249157283479e-05
Не отвергаем нулевую гипотезу, pvalue =  2.067249157283479e-05


### 3) Критикам больше нравятся стрелялки или стратегии?

In [42]:
print("Нулевая гипотеза: Критики одиаково относятся к стрелялкам и стратегиям\nАльтернативная гипотеза: Критики не одиаково относятся к стрелялкам и стратегиям\nУровень статистической значимости 0.05")
Shooter = games[(games['Genre'] == 'Shooter')]
Strategy = games[(games['Genre'] == 'Strategy')]
statistic, pvalue = stats.ttest_ind(Shooter['Critic_Score'], Strategy['Critic_Score'], equal_var=False)

print(statistic,pvalue)
if(pvalue < 0.05):
    print("Не отвергаем нулевую гипотезу, pvalue = ", pvalue)
else:
    print("Отвергаем нулевую гипотезу, pvalue = ", pvalue)

Нулевая гипотеза: Критики одиаково относятся к стрелялкам и стратегиям
Альтернативная гипотеза: Критики не одиаково относятся к стрелялкам и стратегиям
Уровень статистической значимости 0.05
-2.2972408230640315 0.021938989522304823
Не отвергаем нулевую гипотезу, pvalue =  0.021938989522304823


## Задание 2
Реализуйте базовую модель логистической регрессии для классификации текстовых сообщений (используемые данные здесь) по признаку спама. Для этого:

1) Привидите весь текст к нижнему регистру;

2) Удалите мусорные символы;

3) Удалите стоп-слова;

4) Привидите все слова к нормальной форме;

5) Преобразуйте все сообщения в вектора TF-IDF. 

6) Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите random_state=42. Используйте train_test_split;

7) Постройте модель логистической регрессии, укажите random_state=42, оцените ее точность на тестовых данных;

8) Опишите результаты при помощи confusion_matrix;

9) Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного).

In [55]:
spam = pd.read_csv('spam.csv')
#Предобработка текста.
#Удаление мусорных символов, разбиение на слова, удаление стоп слов, приведение к нормальной форме, все в нижнем регистре
spam.Message = spam.Message.str.lower()
spam.Message = spam.Message.apply(lambda x: re.sub('[\W_]+',' ', x))
spam.Words = spam.Message.apply(lambda x: x.split())
stopwords_set = set(stopwords.words('english'))
spam.Words = spam.Words.apply(lambda x: [word for word in x if word not in stopwords_set])
wordnet_lemmatizer = WordNetLemmatizer()
spam.Words = spam.Words.apply(lambda x: [wordnet_lemmatizer.lemmatize(word) for word in x])
spam.Words = spam.Words.apply(lambda x: ' '.join(x))
spam.head()

C:\Users\nbor9\AppData\Local\Temp/ipykernel_16752/4197259489.py:8: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  spam.Words = spam.Message.apply(lambda x: x.split())


,Category,Message
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i don t think he goes to usf he lives arou...


In [56]:
# Преобразуем все сообщения в вектора TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(spam.Words)
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)
tfidf_matrix.head()

,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,02,...,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,èn,ú1,〨ud
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
X = tfidf_matrix
y = spam.Category
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,  random_state=42)
model = LogisticRegression( random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_df = pd.Series(y_pred)
y_pred_df.index = y_test.index
result = pd.concat([y_test, y_pred_df], axis=1)
result.columns = ['y_test', 'y_pred']
display(result)

,y_test,y_pred
3152,ham,ham
4664,ham,ham
5051,ham,ham
1981,ham,ham
3348,ham,ham
...,...,...
537,ham,ham
4307,ham,ham
920,ham,ham
2821,spam,ham


In [60]:
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.968609865470852
[[968   1]
 [ 34 112]]


In [62]:
#Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного).
spam2 = pd.read_csv('spam.csv')

result[(result['y_test'] != result['y_pred'])].join(spam2.drop('Category', axis=1), how='left')


,y_test,y_pred,Message
1050,spam,ham,18 days to Euro2004 kickoff! U will be kept in...
415,spam,ham,100 dating service cal;l 09064012103 box334sk38ch
2430,spam,ham,Guess who am I?This is the first time I create...
4410,spam,ham,For your chance to WIN a FREE Bluetooth Headse...
1718,spam,ham,WOW! The Boys R Back. TAKE THAT 2007 UK Tour. ...
2915,spam,ham,Sorry! U can not unsubscribe yet. THE MOB offe...
4394,spam,ham,RECPT 1/3. You have ordered a Ringtone. Your o...
3574,spam,ham,You won't believe it but it's true. It's Incre...
672,spam,ham,"SMS. ac sun0819 posts HELLO:""You seem cool, wa..."
2300,spam,ham,Congrats! 1 year special cinema pass for 2 is ...
